In [9]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Tag
import requests
from io import StringIO
from typing import Dict, List, Tuple
import json
import re



def pfr_web(letter: str) -> str:
  assert(len(letter) > 0)
  return f"https://www.pro-football-reference.com/players/{letter[0].upper()}/"

def pfr_player_url(player_id: str, year: int = 2024) -> str:
  return f"https://www.pro-football-reference.com/players/{player_id[0].upper()}/{player_id}/gamelog/{year}"

In [7]:
res = requests.get(pfr_web('a'))
html = BeautifulSoup(res.text)
links: List[Tag] = html.find("div", {"id": "all_players"}).find_all("a")

In [11]:
re.findall(r"\((.*?)\)", links[0].next_sibling.strip())

['NT']

In [14]:
def map_ids(year: int = 2024) -> Tuple[Dict[str, str], Dict[str, Dict[str, str | List[str]]]]:
  name_to_id = {}
  id_to_player = {}
  for i in range(0, 26):
    letter = chr(ord('a') + i)
    res = requests.get(pfr_web(letter.upper()))
    if res.status_code != 200: continue
    html = BeautifulSoup(res.text)
    links: List[Tag] = html.find("div", {"id": "all_players"}).find_all("a")
    # res.close()
    for link in links:
      player_name = link.text
      player_id = link.get('href').split('/')[-1].split('.')[0]
      
      # player_res = requests.get(pfr_player_url(player_id, year))
      # player_html = BeautifulSoup(player_res.text)
      # pos = player_html.find("strong", string="Position").next_sibling.split(':')[-1].strip()
      # player_res.close()
      pos = re.findall(r"\((.*?)\)", link.next_sibling.strip())[0]

      name_to_id[player_name] = player_id
      id_to_player[player_id] = {
        "name": player_name,
        "pos": pos,
      }
  return name_to_id, id_to_player

In [13]:
name_to_id, id_to_player = map_ids()

In [29]:
def export(to_player, to_id):
  with open("id_to_player.json", 'w') as f:
    json.dump(id_to_player, f, indent=2)
  with open("name_to_id.json", 'w') as f:
    json.dump(name_to_id, f, indent=2)